In [1]:
import glob
import os

import pandas as pd
from pptx import Presentation
from pptx.util import Inches

from data_analysis.utils import Utils

In [2]:
utils = Utils(
    is_normal=True,
    is_previous=False,
    data_type="spectrogram",
    fit_pos="middle",
    stride="middle",
    kernel_size=128,
    model_type="enn",
    cleansing_type="noting",
)

In [3]:
pptx_filepath = os.path.join(os.environ["git"], "sleep_study", "gallery", "blue_template.pptx")
ss_path = os.path.join(os.environ["git"], "sleep_study", "nidan_enn", "140703_Li", "ss.png")
cm_path = os.path.join(os.environ["git"], "sleep_study", "nidan_enn", "140703_Li", "cm.png")
metrics_path = os.path.join(os.environ["git"], "sleep_study", "nidan_enn", "140703_Li", "metrics.csv")
# img_path = os.path.join(os.environ["git"], "sleep_study", "gallery", "my_network.png")
prs = Presentation(pptx_filepath)

In [21]:
# 0から始まる(0はタイトルスライド)
title_slide_layout = prs.slide_layouts[8]
slide = prs.slides.add_slide(title_slide_layout)
# 睡眠段階を最初のプレースホルダに入れる
placeholder_pict_ss = slide.placeholders[10]
picture = placeholder_pict_ss.insert_picture(ss_path)
picture.crop_top = 0
picture.crop_left = 0
picture.crop_bottom = 0
picture.crop_right = 0
# 混同行列を二個目のプレースホルダに入れる
placeholder_pict_cm = slide.placeholders[11]
picture = placeholder_pict_cm.insert_picture(cm_path)
picture.crop_top = 0
picture.crop_left = 0
picture.crop_bottom = 0
picture.crop_right = 0
# 表を3個目のプレースホルダに入れる
placeholder_table = slide.placeholders[12]
graphic_frame = placeholder_table.insert_table(rows=6, cols=4)
table = graphic_frame.table
df = pd.read_csv(metrics_path)
rec = df.iloc[0, 1:6]
pre = df.iloc[1, 6:11]
f_m = df.iloc[2, 11:]
ss_label = ["nr34", "nr2", "nr1", "rem", "wake"]
rec = rec.rename({"rec_"+_ss_label:_ss_label for _ss_label in ss_label})
pre = pre.rename({"pre_"+_ss_label:_ss_label for _ss_label in ss_label})
f_m = f_m.rename({"f_m_"+_ss_label:_ss_label for _ss_label in ss_label})
df = pd.concat([rec, pre, f_m], axis=1)
df = df.rename(columns={0:"pre", 1:"rec", 2:"f-m"})
# 表の中身の挿入
for row_id in range(1, 6):
    for col_id in range(1, 4):
        table.cell(row_id, col_id).text = f"{df.iloc[row_id-1, col_id-1]:.2f}"
# 行名の挿入
row_label = ["sleep stage"]+ss_label
for i in range(6):
    table.cell(i, 0).text = row_label[i]
# 列名の挿入
col_label = ["rec", "pre", "f-m"]
for i in range(1, 4):
    table.cell(0, i).text = col_label[i-1]
prs.save(os.path.join(os.path.split(pptx_filepath)[0], "output.pptx"))